In [40]:
import pandas as pd
import numpy as np
from datetime import datetime

In [153]:
#Get ETL_DATE for Incremental Ingestion (enabled if ETL_DATE set to 'CURRENT_DATE')
from dotenv import load_dotenv
from pathlib import Path
import os

dotenv_path = Path('db_credentials.env')
load_dotenv(dotenv_path=dotenv_path)

ETL_DATE = os.getenv('ETL_DATE')

In [155]:
# For Incremental Ingestion (enabled if ETL_DATE set to 'CURRENT_DATE')
if ETL_DATE == 'CURRENT_DATE':
    ETL_DATE = datetime.today().strftime('%Y%m%d')
else:
   ETL_DATE = '20250322'
    
print(ETL_DATE)

20250328


In [114]:
df_raw = pd.read_csv(f"data/bronze_{ETL_DATE}_Airline_Reviews_withImageUrls.csv")
print(len(df_raw))
df_raw.head()

52


,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,Aer Lingus,2,“terrible customer service”,26th March 2025,False,Absolutely terrible customer service. Tried t...,NaN,NaN,Couple Leisure,...,Paris to Dublin,March 2025,2.0,3.0,3.0,1.0,NaN,3.0,1,no
1,1,Air France,10,"""excellent customer service""",28th March 2025,True,My son was travelling on Kids Solo as an un...,NaN,NaN,Family Leisure,...,Dublin to Singapore,March 2025,NaN,NaN,NaN,5.0,NaN,NaN,5,yes
2,2,Air India,3,“it was not a pleasant experience”,26th March 2025,False,Recently travelled on Air India flight from T...,NaN,NaN,Family Leisure,...,Toronto to New Delhi,March 2025,1.0,3.0,1.0,4.0,1.0,1.0,1,no
3,3,Air India Express,1,"""not accountable for any damage""",23rd March 2025,True,No proper communication and very casual app...,https://www.airlinequality.com/wp-content/uplo...,NaN,Family Leisure,...,Cochin to Delhi,March 2025,1.0,1.0,1.0,1.0,1.0,1.0,1,no
4,4,AirAsia X,2,"""flight is delay for 20 hours""",24th March 2025,True,"Worse experience, my flight suppose to depa...",NaN,NaN,Business,...,Amritsar to Perth via Kuala Lumpur,March 2025,1.0,1.0,1.0,1.0,NaN,NaN,1,no


#### Rename "Unnamed: 0" to "RowId"

In [117]:
df = df_raw.copy()
df.rename(columns={"Unnamed: 0": "RowId"}, inplace=True)

#### Format and Convert Dates

In [120]:
df['Review Date'] = pd.to_datetime(df['Review Date'], format="mixed")
df['Date Flown'] = pd.to_datetime(df['Date Flown'], format="mixed")
print(len(df))
df.head()

52


,RowId,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,Aer Lingus,2,“terrible customer service”,2025-03-26,False,Absolutely terrible customer service. Tried t...,NaN,NaN,Couple Leisure,...,Paris to Dublin,2025-03-01,2.0,3.0,3.0,1.0,NaN,3.0,1,no
1,1,Air France,10,"""excellent customer service""",2025-03-28,True,My son was travelling on Kids Solo as an un...,NaN,NaN,Family Leisure,...,Dublin to Singapore,2025-03-01,NaN,NaN,NaN,5.0,NaN,NaN,5,yes
2,2,Air India,3,“it was not a pleasant experience”,2025-03-26,False,Recently travelled on Air India flight from T...,NaN,NaN,Family Leisure,...,Toronto to New Delhi,2025-03-01,1.0,3.0,1.0,4.0,1.0,1.0,1,no
3,3,Air India Express,1,"""not accountable for any damage""",2025-03-23,True,No proper communication and very casual app...,https://www.airlinequality.com/wp-content/uplo...,NaN,Family Leisure,...,Cochin to Delhi,2025-03-01,1.0,1.0,1.0,1.0,1.0,1.0,1,no
4,4,AirAsia X,2,"""flight is delay for 20 hours""",2025-03-24,True,"Worse experience, my flight suppose to depa...",NaN,NaN,Business,...,Amritsar to Perth via Kuala Lumpur,2025-03-01,1.0,1.0,1.0,1.0,NaN,NaN,1,no


#### EDA & Check for Nulls (especially for needed key fields)

In [123]:
# Examine nulls (first pass)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   RowId                   52 non-null     int64         
 1   Airline Name            52 non-null     object        
 2   Overall_Rating          52 non-null     int64         
 3   Review_Title            52 non-null     object        
 4   Review Date             52 non-null     datetime64[ns]
 5   Verified                52 non-null     bool          
 6   Review                  52 non-null     object        
 7   Top Review Image Url    8 non-null      object        
 8   Aircraft                15 non-null     object        
 9   Type Of Traveller       52 non-null     object        
 10  Seat Type               52 non-null     object        
 11  Route                   52 non-null     object        
 12  Date Flown              52 non-null     datetime64[n

In [125]:
# Replace any empty string columns with actual nulls
df = df.replace(r'^\s+$', np.nan, regex=True)

#### Replace Overall_Rating 'n' with null, Cast as float

In [128]:
df[df['Overall_Rating'] == 'n']
df['Overall_Rating'] = df['Overall_Rating'].replace('n', np.nan).astype(float)

In [130]:
df[df['Overall_Rating'] == 'n']

,RowId,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended


#### Add Unique Id

In [133]:
def generate_hash(row):
    value = f"{row['Airline Name']}|{row['Review_Title']}|{row['Review Date']}"
    return hashlib.sha256(value.encode()).hexdigest()

In [135]:
df['Id'] = df.apply(generate_hash, axis=1)

In [137]:
# Re-examine nulls (after converting empty string to nulls)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   RowId                   52 non-null     int64         
 1   Airline Name            52 non-null     object        
 2   Overall_Rating          52 non-null     float64       
 3   Review_Title            52 non-null     object        
 4   Review Date             52 non-null     datetime64[ns]
 5   Verified                52 non-null     bool          
 6   Review                  52 non-null     object        
 7   Top Review Image Url    8 non-null      object        
 8   Aircraft                15 non-null     object        
 9   Type Of Traveller       52 non-null     object        
 10  Seat Type               52 non-null     object        
 11  Route                   52 non-null     object        
 12  Date Flown              52 non-null     datetime64[n

Key columns needed do not have nulls

#### Persist to CSV file

In [141]:
df.to_csv(f"data/silver_{ETL_DATE}_Airline_Reviews_Preprocessed.csv", index=False, date_format='%Y-%m-%d')